## Nonparametric modeling and histograms

When we don't need a parametrized description (usually an analytic function with free parameters) of a data set, nonparametric methods offer an alternative approach. The term "nonparametric" doesn't mean that there are no parameters.

- A histogram is one of the most simple nonparametric methods to analyze a one-dimensional data set. To create a histogram, we need to specify bin boundaries, and we implicitly assume that the estimated distribution is piecewise constant within each bin. 

  - Therefore, there are parameters to be determined here as well $-$ the value of the distribution function in each bin.

However, there is no specific distribution class, such as the set of all possible Gaussians or Laplacians, but rather a general set of *distribution-free* models, called the **Sobolev space**. The Sobolev space includes all functions, $h(x)$, that satisfy some smoothness criteria, such as

$$\int [h^{\prime\prime}(x)]^2dx < \infty .$$

This constraint, for example, excludes all functions with infinite spikes. Formally, a method is nonparametric if it provides a distribution function estimate $f(x)$ that approaches the true distribution $h(x)$ with enough data for any $h(x)$ in a class of functions with relatively weak assumptions, such as the Sobolev space above.

- Nonparametric methods play a central role in modern machine learning. They provide the highest possible predictive accuracies, as they can model any distribution shape down to the finest detail, which still has predictive power. However, they typically come at a higher computational cost than more traditional multivariate statistical methods. In addition, it is harder to interpret the results of nonparametric methods than those of parametric models.

### Histograms

Histograms can fit practically any distribution shape, given enough bins. This is essential - each bin can be thought of as a constant estimator of the density in that bin, and the overall histogram as a piecewise constant estimator with a tuning parameter - the number of bins. As the number of data points grows, the number of bins should also grow to capture the increasing amount of detail in the distribution's shape that having more data points allows. This is a general feature of nonparametric methods — they are composed of simple pieces, and the number of pieces grows with the number of data points.


Getting the number of bins right is important. Intuitively, we expect a large bin width to destroy fine-scale features in the data distribution, while a small width will result in increased counting noise per bin. We emphasize that it is not necessary to bin the data before estimating model parameters. 

- A simple example is the case of data drawn from a Gaussian distribution. We can estimate its parameters $\mu$ and $\sigma$ using $ \overline{x} = \frac{1}{N}\sum^N_{i=1}x_i $ and $ s = \sqrt{\frac{1}{N-1}\sum^N_{i=1}(x_i-\overline{x})^2} $ without ever binning the data. 

Nevertheless, binning can allow us to visualize our data and explore various features in order to motivate the model selection.

We will now look at a few rules of thumb for the question of choosing the critical bin width, based on frequentist analyses. Various proposed methods for choosing optimal bin width typically suggest a value proportional to some estimate of the distribution's scale, and decreasing with the sample size. The most popular choice is **“Scott’s rule”** which prescribes a bin width

$$ \Delta_b = \frac{3.5\sigma}{N^{1/3}},$$

where $\sigma$ is the sample standard deviation, and $N$ is the sample size. This rule asymptotically minimizes the mean integrated square error and assumes that the underlying distribution is Gaussian. An attempt to generalize this rule to non-Gaussian distributions is the **Freedman-Diaconis** rule,

$$ \Delta_b = \frac{2(q_{75} - q_{25})}{N^{1/3}} = \frac{2.7\sigma_G}{N^{1/3}} ,$$

which estimates the scale (“spread”) of the distribution from its interquartile range. In the case of a Gaussian distribution, Scott’s bin width is 30% larger than the Freedman–Diaconis bin width.

Although the Freedman–Diaconis rule attempts to account for non-Gaussian distributions, it is too simple to distinguish, for example, multimodal and unimodal distributions that have the same $\sigma_G$. 

- The main reason why finding the optimal bin size is not straightforward is that the result depends on both the actual data distribution and the choice of metric (such as the mean square error) to be optimized. 

The interpretation of binned data essentially represents a model fit, where the model is a piecewise constant function. Different bin widths correspond to different models, and choosing the best bin width amounts to the selection of the best model. The model selection is a topic discussed in detail in chapter 5 on Bayesian statistical inference, and in that context we will describe a powerful method that is cognizant of the detailed properties of a given data distribution. We will also compare these three different rules using multimodal and unimodal distributions (see $\S$5.7.2)

#### Python Implementation of Histograms

`NumPy` and `Matplotlib` contain powerful tools for creating histograms in one dimension or multiple dimensions. The `Matplotlib` command `pylab.hist` is the easiest way to plot a histogram. For more details, see the source code for the many figures in this chapter which show histograms. For computing but not plotting a histogram, the functions `numpy.histogram`, `numpy.histogram2d`, and `numpy.histogramdd` provide optimized implementations. 

The above rules of thumb for choosing bin widths are implemented in the submodule `astropy.stats`, using the functions `knuth_bin_width`, `scott_bin_width`, and `freedman_bin_width`. There is also a pylab-like interface for simple histogramming. The hist function in Astropy operates just like the `hist` function in `Matplotlib`, but can optionally use one of the above routines to choose the binning. For more details see the source code associated with figure 5.20, and the associated discussion in §5.7.2.

In [2]:
# %pylab
# import numpy as np
# x = np.random.normal(size=1000)
# plt.hist(x, bins=50)
# counts, bins = np.histogram(x, bins=50)
# from astropy.visualization import hist
# hist(x, bins='freedman') # can also choose 'knuth' or 'scott'

### How to determine the histogram errors?

Assuming that we have selected a bin size, $\Delta_b$, the $N$ values of $x_i$ are sorted into $M$ bins, with the count in each bin $n_k, k = 1, . . . , M$. If we want to express the results as a properly normalized $f(x)$, with the values $f_k$ in each bin, then it is customary to adopt

$$ f_k = \frac{n_k}{\Delta_bN}. $$

The unit for $f_k$ is the inverse of the unit for $x_i$.

Each estimate of $f_k$ comes with some uncertainty. It is customary to assign "error bars" for each $n_k$ equal to $\sqrt{n_k}$ and thus the uncertainty of $f_k$ is

$$\sigma_k = \frac{\sqrt{n_k}}{\Delta_bN} .$$

This practice assumes that $n_k$ are scattered around the true values in each bin ($\mu$) according to a Gaussian distribution, and that error bars enclose the 68% confidence range for the true value. However, when counts are low this assumption of Gaussianity breaks down and the Poisson distribution should be used instead. 

- For example, according to the Gaussian distribution, negative values of $\mu$ have nonvanishing probability for small $n_k$ (if $n_k = 1$, this probability is 16%). This is clearly wrong since in counting experiments, $\mu \geq 0$. Indeed, if $n_k \geq 1$, then even $\mu = 0$ is clearly ruled out. Note also that $n_k = 0$ does not necessarily imply that $μ = 0$: even if $\mu = 1$, counts will be zero in $1/e \approx 37$% of cases. Another problem is that the range $n_k \pm \sigma_k$ does not correspond to the 68% confidence interval for true $\mu$ when $n_k$ is small. These issues are important when fitting models to small count data (assuming that the available data are already binned). This idea is explored in a Bayesian context in $\S$5.6.6.

## Selection effects and luminosity function estimation

We will now consider the effects of truncated and censored data sets in more detail and introduce a nonparametric methods for correcting the effects of the selection function on the inferred properties of the underlying pdf.

When the selection probability, or selection function $S(x)$ is known (often based on analysis of simulated data sets) and finite, we can use it to correct our estimate $f(x)$. The correction is trivial in the strictly one-dimensional case: the implied true distribution $h(x)$ is obtained from the observed $f(x)$ as

$$ h(x) = \frac{f(x)}{S(x)} $$

When additional observables are available, they might carry additional information about the behavior of the selection function, $S(x)$. One the most important examples in Astronomy is the case of flux-limited sample, as follows.

- Assume that in addition to $x$, we also measure a quantity $y$, and that our selection function is such that $S(x) =1$ for $ y > y_\text{max}(x)$, with $x_\text{min} \leq x \leq x_\text{max} $. Here, the observable $y$ may or may not, be related to observable $x$, and the $y \geq 0$ 

In an astronomical context, $x$ can be thought of as absolute magnitude (a logarithmic measure of luminosity $L$) and $y$ as distance (or redshift in the cosmological context). The differential distribution of luminosity (probability density function) is called the luminosity function. In this example, and for noncosmological distances, we can compute $y_\text{max}(x) = (x/(4\pi F_\text{min}))^{1/2}$, where $F_\text{min}$ is the smallest flux that our measuring apparatus can detect. The observed distribution of our $x$ values is in general different from the distribution we would observe when $S(x) = 1$ for $y \leq (x_\text{max} / 4 \pi F_\text{min}))^{1/2} $ that is, when the "missing region, defined by $y_\text{max} < y \leq (x_\text{max}/ 4 \pi F_\text{min}))^{1/2} = y_\text{max}(x_\text{max}) $, is not excluded. If the two-dimensional probability density is $n(x,y)$, then the latter is given by

$$ h(x) = \int^{y_\text{max}(x_\text{max})}_0 n(x,y) \: dy$$

and the observed distribution corresponds to 

$$ f(x) = \int^{y_\text{max}(x)}_{0} n(x,y) \: dy $$

We can see that the dependence of n(x,y) on $y$ directly affects the difference between $f(x)$ and $h(x)$. Therefore, if we want to obtain an estimate of $h(x)$ based on measurements of $f(x)$, we need to estimate $n(x,y)$ first. Using the same example, $n(x,y)$ is the probability of density function per unit luminosity and unit distance. Note that there is no guarantee that the luminosity function is the same for near and far distances, that is, $n(x,y)$ need not be separable function of $x$ and $y$.

We can formulate the problem as follows. 

- Given a set of measured pairs $(x_i, y_i)$, with $i = 1,...,N$, and known relation $y_\text{max}(x)$ estimate the two-dimensional distribution, $n(x,y)$ from which the sample was drawn. 

- Assume that the measurement errors for both $x$ and $y$ are neglidgeable compared to their observed ranges, that $x$ is measured within a range defined by $x_\text{min}$ and $x_\text{max}$, and that the selection fucntion is 1 for $o \leq y \leq y_\text{max}$ and 0 otherwise. 

### Lynden-Bell's $C^-$ method
Lynden-Bell's nonparametric $C^−$ method can be applied to the above problem when the distributions along the two coordinates $x$ and $y$ are uncorrelated, that is, when we can assume that the bivariate distribution $n(x, y)$ is separable:

$$ n(x,y) = \Psi(x)\rho(y) $$

Therefore, before using the $C^−$ method we need to demonstrate that this assumption is valid.

Following Lynden-Bell, the basic steps for testing that the bivariate distribution n(x, y) is separable are the following:

1. Define a comparable or associated set for each object $i$ such that $J_i = \{j : x_j < x_i, y_j < y_\text{max}(x_i)\}$; this is the largest $x$-limited and $y$-limited data subset for object $i$, with $N_i$ elements (see the left panel of figure 4.8).


2. Sort the set $J_i$ by $y_j$; this gives us the rank $R_j$ for each object (ranging from 1 to $N_i$).


3. Define the rank $R_i$ for object $i$ in its associated set: this is essentially the number of objects
with $y<y_i$ in set $J_i$.
         
         
4. Now, if $x$ and $y$ are truly independent, $R_i$ must be distributed uniformly between 0 and $N_i$; in this case, it is trivial to determine the expectation value and variance for $R_i: E(R_i) = E_i = N_i/2$ and $V(R_i) = V_i = N_i^2/12$. We can define the statistic

$$ \tau = \frac{\sum_i(R_i-E_i)}{\sqrt{\sum_iV_i}} $$

- If $\tau$ < 1, then $x$ and $y$ are uncorrelated at $\sim 1\sigma$ level (this step appears similar to Schmidt’s $V/V_\text{max}$ test discussed below; nevertheless, they are fundamentally different because $V/V_\text{max}$ tests the hypothesis of a uniform distribution in the $y$ direction, while the statistic $\tau$ tests the hypothesis of uncorrelated $x$ and $y$).

The cumulative distributions are defined as

$$ \Phi(x) = \int^x_{-\infty} \Psi (x^\prime)\:dx^\prime $$

and

$$ \Sigma (y) = \int^y_{-\infty} \rho(y^\prime)\:dy^\prime. $$

Then,

$$ \Phi (x_i) = \Phi (x_1)\prod^i_{k=2}(1+1/N_k) $$

where it is assumed that $x_i$ are sorted ($x_1 \leq x_k \leq x_N$). Analogously, if $M_k$ is the number of objects in a set defined by $J_k = \{j:y_j < y_k, y_\text{max}(x_j) > y_k\}$, then

$$ \Sigma(y_j) = \Sigma(y_1) \prod ^j_{k=2}(1+1/M_k) $$

$$ \Phi(x) = \int^x_{-\infty} \Psi (x^\prime)\:dx^\prime $$

$$ \Sigma (y) = \int^y_{-\infty} \rho(y^\prime)\:dy^\prime $$

$$ \Phi (x_i) = \Phi (x_1)\prod^i_{k=2}(1+1/N_k) $$

$$ \Sigma(y_j) = \Sigma(y_1) \prod ^j_{k=2}(1+1/M_k) $$

$$ S_j = \sum_i\bigg(\frac{x_i}{x_\text{max}(j)}\bigg)^3 $$

$$ T_i = \sum^{N_i}_{j=1}\frac{S(x_i,y_j)}{S(x_j,y_j)} $$

$$ R_i = \sum^{N_i}_{j=1} \frac{S(x_i,y_j)}{S(x_j,y_j)} $$
